In [23]:
# null_control
# 1 : null값을 최빈값으로 변경, 
# 2 : null값을 삭제
null_control = 1

# 머신러닝 모델 설명

아래는 다양한 머신러닝 모델들의 역할과 특징을 설명한 내용입니다.  
각 모델은 데이터의 패턴을 학습하여 새로운 데이터를 예측하는 데 사용됩니다.

## 1. 선형 모델 (Linear Models)

### **로지스틱 회귀 (Logistic Regression)**
- 분류 문제를 해결하기 위한 통계 모델
- 시그모이드 함수를 이용하여 이진 분류 수행
- 데이터가 선형적으로 분리될 때 효과적

---

## 2. 거리 기반 모델 (Distance-Based Models)

### **K-최근접 이웃 (K-Nearest Neighbors, KNN)**
- 데이터 포인트 간의 거리를 계산하여 가장 가까운 K개의 데이터를 참조해 분류
- 데이터의 분포가 중요하며, 연산량이 많을 수 있음

---

## 3. 트리 기반 모델 (Tree-Based Models)

### **의사결정 트리 (Decision Tree)**
- 데이터를 트리 구조로 나누어 분류하는 모델
- 해석이 쉽고 직관적이지만, 과적합될 가능성이 있음

### **랜덤 포레스트 (Random Forest)**
- 여러 개의 의사결정 트리를 결합하여 예측 성능을 향상시킨 모델
- 과적합을 방지하면서 높은 성능을 발휘

### **엑스트라 트리 (Extra Trees)**
- 랜덤 포레스트와 유사하지만, 분할 기준을 더욱 무작위로 선택하여 학습 속도를 향상

---

## 4. 부스팅 기반 모델 (Boosting Models)

### **그래디언트 부스팅 (Gradient Boosting)**
- 이전 모델이 예측하지 못한 부분을 보완하면서 점진적으로 향상되는 모델
- 정확도가 높지만, 학습 시간이 오래 걸릴 수 있음

### **에이다부스트 (AdaBoost)**
- 여러 개의 약한 학습기(의사결정 트리)를 조합하여 강력한 분류기를 만듦
- 노이즈가 많은 데이터에서는 성능이 떨어질 수 있음

### **XGBoost (eXtreme Gradient Boosting)**
- 그래디언트 부스팅을 개선하여 빠르고 강력한 모델
- 과적합을 방지하는 정규화 기능이 추가됨 

---

## 5. 확률 기반 모델 (Probability-Based Models)

### **나이브 베이즈 (Naive Bayes)**
- 베이즈 정리를 기반으로 한 확률 모델
- 단어 출현 확률을 이용한 스팸 필터링 등에 자주 사용됨
- 독립 가정이 성립할 때 강력한 성능을 보임

---

## 6. 커널 기반 모델 (Kernel-Based Models)

### **서포트 벡터 머신 (Support Vector Machine, SVM)**
- 초평면을 이용해 데이터를 분류하는 모델
- 데이터가 선형적으로 분리되지 않는 경우에도 커널 트릭을 사용해 분류 가능
- 계산량이 많아 대규모 데이터에는 적합하지 않을 수 있음


In [24]:
# !pip install xgboost

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier

In [56]:
url = "../data/(최종)_서울열선_광진도로.csv"
# url = "../data/(증폭)_서울열선_광진도로.csv"
file_encoding = "UTF-8"

In [57]:
df = pd.read_csv(url, encoding = file_encoding)

In [58]:
df.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역', '고도의_차이', '경사각',
       '최근접_시설들_거리', '최근접_시설의_평균거리', '2019_평균_온도', '2020_평균_온도', '2021_평균_온도',
       '2022_평균_온도', '2023_평균_온도', '2024_평균_온도', '종합_평균_기온', '최근접_시설들_최소거리',
       '최근접_시설들_최대거리'],
      dtype='object')

In [59]:
df['최근접_시설들_최소거리'] = df['최근접_시설들_거리'].apply(lambda x: np.min(list(map(float, x.split(',')))))
df['최근접_시설들_최대거리'] = df['최근접_시설들_거리'].apply(lambda x: np.max(list(map(float, x.split(',')))))

In [60]:
road_width = {
    '폭20-25m' : 22.5,
    '폭15-20m' : 17.5,
    '폭8-10m' : 9.0,
    '폭10-12m' : 11.0,
    '폭6-8m' : 7.0,
    '6m미만' : 5.0
}

df['도로폭'] = df['도로폭'].map(road_width)

In [61]:
road_scale = {
    "소로" : 1,   
    "소로3류" : 2,  
    "소로1류" : 3,
    "소로2류" : 4,
    "중로2류" : 5,
    "중로1류" : 6
}

df['도로규모'] = df['도로규모'].map(road_scale)

In [62]:
road_title = {
    '3차로' : 1,
    '주거지 도로' : 2,
    '생활도로' : 3
}
        
df['도로 종류'] = df['도로 종류'].map(road_title)

In [63]:
print(df['도로폭'].unique())
print(df['도로규모'].unique())
print(df['도로 종류'].unique())

[22.5  7.   5.   9.  11.  17.5]
[6 2 1 4 3 5]
[1 2 3]


In [64]:
if null_control == 1 :
    # 최빈값을 대입
    print(f"널값이 있으면 해당 컬럼의 최빈값으로 값을 대체합니다.")
    for col in ['경사각', '최근접_시설의_평균거리', '도로규모', '도로 종류', '열선', '도로폭',
          '2019_평균_온도', '2020_평균_온도', '2021_평균_온도',
          '2022_평균_온도', '2023_평균_온도', '2024_평균_온도', '종합_평균_기온']:
        df[col] = df[col].fillna(df[col].mode()[0])
if null_control == 2 :
    # 널값 삭제
    print(f"널값이 있으면 해당 데이터는 삭제합니다.")
    df = df.dropna(subset=['경사각', '최근접_시설의_평균거리', '도로규모', '도로 종류', '열선', '도로폭',
          '2019_평균_온도', '2020_평균_온도', '2021_평균_온도',
          '2022_평균_온도', '2023_평균_온도', '2024_평균_온도', '종합_평균_기온'])

널값이 있으면 해당 컬럼의 최빈값으로 값을 대체합니다.


In [65]:
df.isnull().sum()

도로명             0
열선              0
도로 종류           0
시작점_위도          0
시작점_경도          0
종료점_위도          0
종료점_경도          0
중앙점_위도          0
중앙점_경도          0
도로_길이           0
행정동             0
도로폭             0
도로규모            0
행정구역            0
고도의_차이          0
경사각             0
최근접_시설들_거리      0
최근접_시설의_평균거리    0
2019_평균_온도      0
2020_평균_온도      0
2021_평균_온도      0
2022_평균_온도      0
2023_평균_온도      0
2024_평균_온도      0
종합_평균_기온        0
최근접_시설들_최소거리    0
최근접_시설들_최대거리    0
dtype: int64

In [66]:
# 특성과 타겟 변수 분리
X = df[['경사각', '최근접_시설의_평균거리', '도로규모', '도로 종류', '도로폭', 
          '최근접_시설의_평균거리', '최근접_시설들_최소거리', '최근접_시설들_최대거리',
          '2019_평균_온도', '2020_평균_온도', '2021_평균_온도',
          '2022_평균_온도', '2023_평균_온도', '2024_평균_온도', '종합_평균_기온']]
y = df['열선']

In [67]:
# 모델 목록
models = { 
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=3),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Extra Trees': ExtraTreesClassifier(random_state=42),
    'Naive Bayes': GaussianNB()
}


In [68]:
temp_model_names = list(models.keys())
temp_model_names

['SVM',
 'KNN',
 'Decision Tree',
 'Random Forest',
 'Gradient Boosting',
 'AdaBoost',
 'Extra Trees',
 'Naive Bayes']

In [69]:
# 데이터 셔플 및 훈련, 테스트 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle = True)

In [70]:
result_model_name = []
result_model_score = []
result_model_find = []

In [71]:
for model_name, model in models.items():
    # 새로운 데이터 프레임 복사
    new_df = df.copy()

    # 모델 학습 및 예측
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = accuracy_score(y_test, y_pred)

    temp = round(score, 4)
    result_model_name.append(model_name)
    result_model_score.append(temp)

    # 전체 데이터에 대해 최적 모델로 예측값 추가
    result_variable = "최적모델_예측값"
    new_df[result_variable] = model.predict(pd.DataFrame(X))  # 데이터프레임 변환

    # 열선이 있어야 한다고 예측한 데이터만 분리
    filter_df = new_df[new_df[result_variable] == 1]
    temp_result_df = filter_df[filter_df["열선"] == 0]

    # 컬럼 존재 여부 확인
    if not {"중앙점_위도", "중앙점_경도"}.issubset(temp_result_df.columns):
        print(f"오류: {model_name} 모델에서 '중앙점_위도' 또는 '중앙점_경도' 컬럼이 존재하지 않습니다.")
        continue

    # 중복된 (중앙점_위도, 중앙점_경도) 값 찾기
    duplicates = temp_result_df.duplicated(subset=['중앙점_위도', '중앙점_경도'], keep=False)

    # 중복된 값 제거
    result_df = temp_result_df[~duplicates]
    result_model_find.append(len(result_df))

    # 저장 경로 설정
    save_url_1 = "증폭" if "증폭" in url else "원본"
    full_save_url = f"./make_file/({save_url_1})_{model_name}_분석한_결과.csv"

    # CSV 파일 저장
    try:
        result_df.to_csv(full_save_url, encoding=file_encoding, index=False)
        print(f"{full_save_url} 저장 완료")
    except OSError as e:
        print(f"파일 저장 오류: {e}")


./make_file/(원본)_SVM_분석한_결과.csv 저장 완료
./make_file/(원본)_KNN_분석한_결과.csv 저장 완료
./make_file/(원본)_Decision Tree_분석한_결과.csv 저장 완료
./make_file/(원본)_Random Forest_분석한_결과.csv 저장 완료
./make_file/(원본)_Gradient Boosting_분석한_결과.csv 저장 완료
./make_file/(원본)_AdaBoost_분석한_결과.csv 저장 완료
./make_file/(원본)_Extra Trees_분석한_결과.csv 저장 완료
./make_file/(원본)_Naive Bayes_분석한_결과.csv 저장 완료


In [72]:
for i in range(len(result_model_find)) :
    print(f"모델 명 : {result_model_name[i]:30s}, ", end = " ")
    print(f"모델 성능 : {result_model_score[i]:.04f}, ", end = " ")
    print(f"찾은 열선 : {result_model_find[i]}곳")

모델 명 : SVM                           ,  모델 성능 : 0.8484,  찾은 열선 : 0곳
모델 명 : KNN                           ,  모델 성능 : 0.8466,  찾은 열선 : 29곳
모델 명 : Decision Tree                 ,  모델 성능 : 0.9593,  찾은 열선 : 13곳
모델 명 : Random Forest                 ,  모델 성능 : 0.9635,  찾은 열선 : 9곳
모델 명 : Gradient Boosting             ,  모델 성능 : 0.9556,  찾은 열선 : 11곳
모델 명 : AdaBoost                      ,  모델 성능 : 0.9339,  찾은 열선 : 22곳
모델 명 : Extra Trees                   ,  모델 성능 : 0.9653,  찾은 열선 : 11곳
모델 명 : Naive Bayes                   ,  모델 성능 : 0.8919,  찾은 열선 : 68곳
